In [ ]:
pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.5 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import datetime

import psycopg2
import pymysql

from datetime import timedelta

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
host = '157.245.248.249'
port = int(5432)
dbname = 'ubuntu'
user = 'ubuntuuser'
password = '97cD8$pt@'


def get_query_results_postgres():
    with psycopg2.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = "SELECT * FROM bloomlive.loans_fact_table_summary_view"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()

In [ ]:
loans = get_query_results_postgres()

loans.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,safaricom_loan_balance
0,1853185,16296,2.000,102569,14269,None,71,2,NaN,NaN,600,000016296,Individual Loan,mifos-standard-strategy,7,Days,7,4300.000,4300.000,4300.000,4300.000,0.000,0.000,146.210,146.210,0.000,0.000,0.000,0.010,0.010,0.000,0.000,0.000,0.050,0.050,0.000,0.000,0.000,4446.270,4446.270,146.270,146.270,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-20,2022-01-20,2022-01-20,2022-01-20,2022-01-27,2022-01-27,None,2022-01-27,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7529969,3.000,None,QAK6VXQGQY,4300.000,177.286,47,3.000,NaN,12,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680510,NaT,399.000,4300.000,4300.000,0.000,NaN,10002,2022-01-30,2022-03-01,2022-03-31,2022-04-30,NaN
1,1853244,16357,2.000,90739,2439,None,72,3,NaN,NaN,600,000016357,Individual Loan,mifos-standard-strategy,21,Days,21,56000.000,56000.000,56000.000,56000.000,0.000,0.000,4872.010,4872.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60872.010,60872.010,4872.010,4872.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-21,2022-01-21,2022-01-21,2022-01-21,2022-02-11,2022-02-11,None,2022-02-11,None,None,None,None,None,None,False,None,None,False,None,False,False,None,840549,3.000,None,QAL6WDCNS8,56000.000,151.214,47,3.000,NaN,6,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680530,NaT,NaN,56000.000,56000.000,0.000,NaN,10003,2022-02-16,2022-03-18,2022-04-17,2022-05-17,NaN


In [ ]:
loans = loans[loans['bloom_version'] == 2]

loans = loans[(loans['loan_status'] == 300) | (loans['loan_status'] == 600)| (loans['loan_status'] == 700)]

loans.shape

(198312, 107)

In [ ]:
#create loan count column i.e adds a new column that captures the num of loans a customer has taken
loans["loan_count"] = loans.groupby("store_number")["store_number"].transform('size')

In [ ]:
host = '167.71.47.216'
port = int(3306)
user = 'tableau_reports'
password = 'change@123'
dbname = 'mifostenant-safaricom'

def get_query_results_mysql():
    with pymysql.connect(host = host,
                         port = port,
                         database = dbname,
                         user = user,
                         password = password) as conn:
        sql = "SELECT * FROM m_loan_transaction"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()

In [ ]:
df_transactions = get_query_results_mysql()

df_transactions.head(2)

,id,loan_id,office_id,payment_detail_id,is_reversed,external_id,transaction_type_enum,transaction_date,amount,principal_portion_derived,interest_portion_derived,fee_charges_portion_derived,penalty_charges_portion_derived,overpayment_portion_derived,unrecognized_income_portion,outstanding_loan_balance_derived,submitted_on_date,manually_adjusted_or_reversed,created_date,appuser_id
0,13999,3697,1,7245.000,0,None,1,2021-11-05,150.000,NaN,NaN,NaN,NaN,NaN,None,150.000,2021-12-27,0,2021-12-27 12:26:34,1
1,14000,3697,1,NaN,0,None,10,2021-11-05,5.090,NaN,5.090,NaN,NaN,NaN,None,NaN,2021-12-27,0,2021-12-27 12:26:34,1


In [ ]:
# filtering df to only contain repayments
df_transactions = df_transactions[(df_transactions['transaction_type_enum'] == 2) & (df_transactions['is_reversed'] == 0)]

df_transactions.shape

(375172, 20)

In [ ]:
df_transactions.head(2)

,id,loan_id,office_id,payment_detail_id,is_reversed,external_id,transaction_type_enum,transaction_date,amount,principal_portion_derived,interest_portion_derived,fee_charges_portion_derived,penalty_charges_portion_derived,overpayment_portion_derived,unrecognized_income_portion,outstanding_loan_balance_derived,submitted_on_date,manually_adjusted_or_reversed,created_date,appuser_id
100,14099,3698,1,7295.000,0,None,2,2021-11-05,30.000,21.310,8.690,NaN,NaN,NaN,None,78.690,2021-12-27,0,2021-12-27 13:21:32,1
101,14100,3703,1,7296.000,0,None,2,2021-11-05,111.000,102.000,8.870,NaN,NaN,0.130,None,0.000,2021-12-27,0,2021-12-27 13:21:41,1


In [ ]:
#renaming loan_id to loan_mifos_id
df_transactions.rename(columns = {'loan_id':'loan_mifos_id'}, inplace = True)

# grouping loan_mifos_id to find the max transaction date
transactions = df_transactions.groupby(['loan_mifos_id'], as_index=False)['transaction_date'].max()
transactions = transactions.rename(columns={'transaction_date':'max_transaction_date'})

transactions.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,loan_mifos_id,max_transaction_date
0,3697,2021-11-05
1,3698,2021-11-05


In [ ]:
transactions.shape

(186027, 2)

In [ ]:
#changing the dates column to datetime datatype
loans['disbursed_on_date'] = pd.to_datetime(loans['disbursed_on_date'])

loans['expected_matured_on_date'] = pd.to_datetime(loans['expected_matured_on_date'])

#changing the dates column to datetime datatype
transactions['max_transaction_date'] = pd.to_datetime(transactions['max_transaction_date'])

In [ ]:
transactions.shape

(186027, 2)

In [ ]:
# merging df_transactions with defaulters
df = pd.merge(loans, transactions, on = "loan_mifos_id", how = "left")
df.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,safaricom_loan_balance,loan_count,max_transaction_date
0,1853185,16296,2.000,102569,14269,None,71,2,NaN,NaN,600,000016296,Individual Loan,mifos-standard-strategy,7,Days,7,4300.000,4300.000,4300.000,4300.000,0.000,0.000,146.210,146.210,0.000,0.000,0.000,0.010,0.010,0.000,0.000,0.000,0.050,0.050,0.000,0.000,0.000,4446.270,4446.270,146.270,146.270,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-20,2022-01-20,2022-01-20,2022-01-20,2022-01-27,2022-01-27,None,2022-01-27,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7529969,3.000,None,QAK6VXQGQY,4300.000,177.286,47,3.000,NaN,12,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680510,NaT,399.000,4300.000,4300.000,0.000,NaN,10002,2022-01-30,2022-03-01,2022-03-31,2022-04-30,NaN,8,2022-01-27
1,1853244,16357,2.000,90739,2439,None,72,3,NaN,NaN,600,000016357,Individual Loan,mifos-standard-strategy,21,Days,21,56000.000,56000.000,56000.000,56000.000,0.000,0.000,4872.010,4872.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60872.010,60872.010,4872.010,4872.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-21,2022-01-21,2022-01-21,2022-01-21,2022-02-11,2022-02-11,None,2022-02-11,None,None,None,None,None,None,False,None,None,False,None,False,False,None,840549,3.000,None,QAL6WDCNS8,56000.000,151.214,47,3.000,NaN,6,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680530,NaT,NaN,56000.000,56000.000,0.000,NaN,10003,2022-02-16,2022-03-18,2022-04-17,2022-05-17,NaN,14,2022-02-11


In [ ]:
df.shape

(198312, 109)

In [ ]:
#create due_date fixed column to ensure due date tallies with the term frequency for each loan

df["due_date_fixed"] = df["disbursed_on_date"] + pd.to_timedelta(df["term_frequency"], unit='d')

df['due_date_fixed'] = pd.to_datetime(df['due_date_fixed'], errors = 'coerce')

df.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,safaricom_loan_balance,loan_count,max_transaction_date,due_date_fixed
0,1853185,16296,2.000,102569,14269,None,71,2,NaN,NaN,600,000016296,Individual Loan,mifos-standard-strategy,7,Days,7,4300.000,4300.000,4300.000,4300.000,0.000,0.000,146.210,146.210,0.000,0.000,0.000,0.010,0.010,0.000,0.000,0.000,0.050,0.050,0.000,0.000,0.000,4446.270,4446.270,146.270,146.270,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-20,2022-01-20,2022-01-20,2022-01-20,2022-01-27,2022-01-27,None,2022-01-27,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7529969,3.000,None,QAK6VXQGQY,4300.000,177.286,47,3.000,NaN,12,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680510,NaT,399.000,4300.000,4300.000,0.000,NaN,10002,2022-01-30,2022-03-01,2022-03-31,2022-04-30,NaN,8,2022-01-27,2022-01-27
1,1853244,16357,2.000,90739,2439,None,72,3,NaN,NaN,600,000016357,Individual Loan,mifos-standard-strategy,21,Days,21,56000.000,56000.000,56000.000,56000.000,0.000,0.000,4872.010,4872.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60872.010,60872.010,4872.010,4872.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-21,2022-01-21,2022-01-21,2022-01-21,2022-02-11,2022-02-11,None,2022-02-11,None,None,None,None,None,None,False,None,None,False,None,False,False,None,840549,3.000,None,QAL6WDCNS8,56000.000,151.214,47,3.000,NaN,6,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680530,NaT,NaN,56000.000,56000.000,0.000,NaN,10003,2022-02-16,2022-03-18,2022-04-17,2022-05-17,NaN,14,2022-02-11,2022-02-11


In [ ]:
df['curr_time'] = '2022-09-12'

df['curr_time'] = pd.to_datetime(df['curr_time'], errors = 'coerce')

df.head()

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,safaricom_loan_balance,loan_count,max_transaction_date,due_date_fixed,curr_time
0,1853185,16296,2.000,102569,14269,None,71,2,NaN,NaN,600,000016296,Individual Loan,mifos-standard-strategy,7,Days,7,4300.000,4300.000,4300.000,4300.000,0.000,0.000,146.210,146.210,0.000,0.000,0.000,0.010,0.010,0.000,0.000,0.000,0.050,0.050,0.000,0.000,0.000,4446.270,4446.270,146.270,146.270,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-20,2022-01-20,2022-01-20,2022-01-20,2022-01-27,2022-01-27,None,2022-01-27,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7529969,3.000,None,QAK6VXQGQY,4300.000,177.286,47,3.000,NaN,12,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680510,NaT,399.000,4300.000,4300.000,0.000,NaN,10002,2022-01-30,2022-03-01,2022-03-31,2022-04-30,NaN,8,2022-01-27,2022-01-27,2022-09-12
1,1853244,16357,2.000,90739,2439,None,72,3,NaN,NaN,600,000016357,Individual Loan,mifos-standard-strategy,21,Days,21,56000.000,56000.000,56000.000,56000.000,0.000,0.000,4872.010,4872.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60872.010,60872.010,4872.010,4872.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-21,2022-01-21,2022-01-21,2022-01-21,2022-02-11,2022-02-11,None,2022-02-11,None,None,None,None,None,None,False,None,None,False,None,False,False,None,840549,3.000,None,QAL6WDCNS8,56000.000,151.214,47,3.000,NaN,6,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680530,NaT,NaN,56000.000,56000.000,0.000,NaN,10003,2022-02-16,2022-03-18,2022-04-17,2022-05-17,NaN,14,2022-02-11,2022-02-11,2022-09-12
2,1853271,16386,2.000,95820,7520,None,71,2,NaN,NaN,600,000016386,Individual Loan,mifos-standard-strategy,7,Days,7,1400.000,1400.000,1400.000,1400.000,0.000,0.000,47.610,47.610,0.000,0.000,0.000,0.010,0.010,0.000,0.000,0.000,0.050,0.050,0.000,0.000,0.

In [ ]:
def calc_days_past_due(df):
        
    res = 0
    loan_status = df['loan_status']
    expected_matured_on_date = df['expected_matured_on_date']
    max_transaction_date = df['max_transaction_date']
    curr_time = df['curr_time']
    
    if (loan_status == 600) and (max_transaction_date > expected_matured_on_date):
        res = (max_transaction_date - expected_matured_on_date).days
    elif (loan_status == 700) and (max_transaction_date > expected_matured_on_date):
        res = (max_transaction_date - expected_matured_on_date).days
    elif (loan_status == 300) and (curr_time > expected_matured_on_date):
        res = (curr_time - expected_matured_on_date).days
        
    return res if res > 0 else res

In [ ]:
df['days_past_due'] = df.apply(lambda x: calc_days_past_due(x), axis = 1)

df.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,safaricom_loan_balance,loan_count,max_transaction_date,due_date_fixed,curr_time,days_past_due
0,1853185,16296,2.000,102569,14269,None,71,2,NaN,NaN,600,000016296,Individual Loan,mifos-standard-strategy,7,Days,7,4300.000,4300.000,4300.000,4300.000,0.000,0.000,146.210,146.210,0.000,0.000,0.000,0.010,0.010,0.000,0.000,0.000,0.050,0.050,0.000,0.000,0.000,4446.270,4446.270,146.270,146.270,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-20,2022-01-20,2022-01-20,2022-01-20,2022-01-27,2022-01-27,None,2022-01-27,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7529969,3.000,None,QAK6VXQGQY,4300.000,177.286,47,3.000,NaN,12,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680510,NaT,399.000,4300.000,4300.000,0.000,NaN,10002,2022-01-30,2022-03-01,2022-03-31,2022-04-30,NaN,8,2022-01-27,2022-01-27,2022-09-12,0
1,1853244,16357,2.000,90739,2439,None,72,3,NaN,NaN,600,000016357,Individual Loan,mifos-standard-strategy,21,Days,21,56000.000,56000.000,56000.000,56000.000,0.000,0.000,4872.010,4872.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60872.010,60872.010,4872.010,4872.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-21,2022-01-21,2022-01-21,2022-01-21,2022-02-11,2022-02-11,None,2022-02-11,None,None,None,None,None,None,False,None,None,False,None,False,False,None,840549,3.000,None,QAL6WDCNS8,56000.000,151.214,47,3.000,NaN,6,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680530,NaT,NaN,56000.000,56000.000,0.000,NaN,10003,2022-02-16,2022-03-18,2022-04-17,2022-05-17,NaN,14,2022-02-11,2022-02-11,2022-09-12,0


In [ ]:
def calc_end_rollover_date(df):
    tenure = df['repay_every']
    expected_matured_on_date = df['expected_matured_on_date']
    
    if tenure == 21:
        res = expected_matured_on_date + pd.to_timedelta(5, unit='d')
    elif tenure == 7:
        res = expected_matured_on_date + pd.to_timedelta(3, unit='d')
    elif tenure == 1:
        res = expected_matured_on_date + pd.to_timedelta(1, unit='d')
        
    return res

In [ ]:
df['end_rollover_date'] = df.apply(lambda x: calc_end_rollover_date(x), axis = 1)

df.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,safaricom_loan_balance,loan_count,max_transaction_date,due_date_fixed,curr_time,days_past_due,end_rollover_date
0,1853185,16296,2.000,102569,14269,None,71,2,NaN,NaN,600,000016296,Individual Loan,mifos-standard-strategy,7,Days,7,4300.000,4300.000,4300.000,4300.000,0.000,0.000,146.210,146.210,0.000,0.000,0.000,0.010,0.010,0.000,0.000,0.000,0.050,0.050,0.000,0.000,0.000,4446.270,4446.270,146.270,146.270,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-20,2022-01-20,2022-01-20,2022-01-20,2022-01-27,2022-01-27,None,2022-01-27,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7529969,3.000,None,QAK6VXQGQY,4300.000,177.286,47,3.000,NaN,12,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680510,NaT,399.000,4300.000,4300.000,0.000,NaN,10002,2022-01-30,2022-03-01,2022-03-31,2022-04-30,NaN,8,2022-01-27,2022-01-27,2022-09-12,0,2022-01-30
1,1853244,16357,2.000,90739,2439,None,72,3,NaN,NaN,600,000016357,Individual Loan,mifos-standard-strategy,21,Days,21,56000.000,56000.000,56000.000,56000.000,0.000,0.000,4872.010,4872.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60872.010,60872.010,4872.010,4872.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-21,2022-01-21,2022-01-21,2022-01-21,2022-02-11,2022-02-11,None,2022-02-11,None,None,None,None,None,None,False,None,None,False,None,False,False,None,840549,3.000,None,QAL6WDCNS8,56000.000,151.214,47,3.000,NaN,6,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680530,NaT,NaN,56000.000,56000.000,0.000,NaN,10003,2022-02-16,2022-03-18,2022-04-17,2022-05-17,NaN,14,2022-02-11,2022-02-11,2022-09-12,0,2022-02-16


In [ ]:
#create days past due date column to ensure due date tallies with the term frequency for each loan

df["dpd_90_date"] = df["end_rollover_date"] + pd.to_timedelta(90, unit='d')

df['dpd_90_date'] = pd.to_datetime(df['dpd_90_date'])

df.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,safaricom_loan_balance,loan_count,max_transaction_date,due_date_fixed,curr_time,days_past_due,end_rollover_date,dpd_90_date
0,1853185,16296,2.000,102569,14269,None,71,2,NaN,NaN,600,000016296,Individual Loan,mifos-standard-strategy,7,Days,7,4300.000,4300.000,4300.000,4300.000,0.000,0.000,146.210,146.210,0.000,0.000,0.000,0.010,0.010,0.000,0.000,0.000,0.050,0.050,0.000,0.000,0.000,4446.270,4446.270,146.270,146.270,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-20,2022-01-20,2022-01-20,2022-01-20,2022-01-27,2022-01-27,None,2022-01-27,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7529969,3.000,None,QAK6VXQGQY,4300.000,177.286,47,3.000,NaN,12,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680510,NaT,399.000,4300.000,4300.000,0.000,NaN,10002,2022-01-30,2022-03-01,2022-03-31,2022-04-30,NaN,8,2022-01-27,2022-01-27,2022-09-12,0,2022-01-30,2022-04-30
1,1853244,16357,2.000,90739,2439,None,72,3,NaN,NaN,600,000016357,Individual Loan,mifos-standard-strategy,21,Days,21,56000.000,56000.000,56000.000,56000.000,0.000,0.000,4872.010,4872.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60872.010,60872.010,4872.010,4872.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-01-21,2022-01-21,2022-01-21,2022-01-21,2022-02-11,2022-02-11,None,2022-02-11,None,None,None,None,None,None,False,None,None,False,None,False,False,None,840549,3.000,None,QAL6WDCNS8,56000.000,151.214,47,3.000,NaN,6,1,NaN,None,None,1.000,1.000,NaN,2022-04-18 19:59:27.068490,110000000000680530,NaT,NaN,56000.000,56000.000,0.000,NaN,10003,2022-02-16,2022-03-18,2022-04-17,2022-05-17,NaN,14,2022-02-11,2022-02-11,2022-09-12,0,2022-02-16,2022-05-17


In [ ]:
df = df[['loan_mifos_id', 'store_number', 'term_frequency', 'principal_disbursed', 'curr_time', 'disbursed_on_date', 'total_expected_repayment', 'total_repayment', 'due_date_fixed', 'expected_matured_on_date', 'max_transaction_date', 'due_date_fixed', 'end_rollover_date', 'dpd_90_date', 'days_past_due']]

df.head(2)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,curr_time,disbursed_on_date,total_expected_repayment,total_repayment,due_date_fixed,expected_matured_on_date,max_transaction_date,due_date_fixed,end_rollover_date,dpd_90_date,days_past_due
0,16296,7529969,7,4300.000,2022-09-12,2022-01-20,4446.270,4446.270,2022-01-27,2022-01-27,2022-01-27,2022-01-27,2022-01-30,2022-04-30,0
1,16357,840549,21,56000.000,2022-09-12,2022-01-21,60872.010,60872.010,2022-02-11,2022-02-11,2022-02-11,2022-02-11,2022-02-16,2022-05-17,0


In [ ]:
df_transactions = df_transactions[['loan_mifos_id', 'transaction_date', 'amount']]

df_transactions.head(2)

,loan_mifos_id,transaction_date,amount
100,3698,2021-11-05,30.000
101,3703,2021-11-05,111.000


In [ ]:
df2 = pd.merge(df, df_transactions, on = 'loan_mifos_id', how = 'left')

df2.head(2)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,curr_time,disbursed_on_date,total_expected_repayment,total_repayment,due_date_fixed,expected_matured_on_date,max_transaction_date,due_date_fixed,end_rollover_date,dpd_90_date,days_past_due,transaction_date,amount
0,16296,7529969,7,4300.000,2022-09-12,2022-01-20,4446.270,4446.270,2022-01-27,2022-01-27,2022-01-27,2022-01-27,2022-01-30,2022-04-30,0,2022-01-27,4446.200
1,16357,840549,21,56000.000,2022-09-12,2022-01-21,60872.010,60872.010,2022-02-11,2022-02-11,2022-02-11,2022-02-11,2022-02-16,2022-05-17,0,2022-02-11,60872.000


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387457 entries, 0 to 387456
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   loan_mifos_id             387457 non-null  int64         
 1   store_number              387457 non-null  object        
 2   term_frequency            387457 non-null  int64         
 3   principal_disbursed       387457 non-null  float64       
 4   curr_time                 387457 non-null  datetime64[ns]
 5   disbursed_on_date         387457 non-null  datetime64[ns]
 6   total_expected_repayment  387457 non-null  float64       
 7   total_repayment           387457 non-null  float64       
 8   due_date_fixed            387457 non-null  datetime64[ns]
 9   expected_matured_on_date  387457 non-null  datetime64[ns]
 10  max_transaction_date      375172 non-null  datetime64[ns]
 11  due_date_fixed            387457 non-null  datetime64[ns]
 12  en

In [ ]:
df2['transaction_date'] = pd.to_datetime(df2['transaction_date'], errors='coerce')

df2['disbursed_on_date'] = pd.to_datetime(df2['disbursed_on_date'], errors='coerce')

df2['expected_matured_on_date'] = pd.to_datetime(df2['expected_matured_on_date'], errors='coerce')

df2['end_rollover_date'] = pd.to_datetime(df2['end_rollover_date'], errors='coerce')

df2['dpd_90_date'] = pd.to_datetime(df2['dpd_90_date'], errors='coerce')

df2['curr_time'] = pd.to_datetime(df2['curr_time'], errors='coerce')

In [ ]:
def calc_trans_by_due_date(df):
    
    expected_matured_on_date = df['expected_matured_on_date']
    transaction_date = df['transaction_date']
    
    if transaction_date <= expected_matured_on_date:
        return 'yes'
    else:
        return 'no'
    
    
def calc_trans_by_end_rollover_date(df):
    
    end_rollover_date = df['end_rollover_date']
    transaction_date = df['transaction_date']
    
    if transaction_date <= end_rollover_date:
        return 'yes'
    else:
        return 'no'
    
    
def calc_trans_by_dpd_90_date(df):
    
    dpd_90_date = df['dpd_90_date']
    transaction_date = df['transaction_date']
    
    if transaction_date <= dpd_90_date:
        return 'yes'
    else:
        return 'no'

In [ ]:
df2['transaction_by_due_date'] = df2.apply(lambda x: calc_trans_by_due_date(x), axis = 1)
df2['transaction_by_end_rollover_date'] = df2.apply(lambda x: calc_trans_by_end_rollover_date(x), axis = 1)
df2['transaction_by_dpd_90_date'] = df2.apply(lambda x: calc_trans_by_dpd_90_date(x), axis = 1)


df2.head(2)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,curr_time,disbursed_on_date,total_expected_repayment,total_repayment,due_date_fixed,expected_matured_on_date,max_transaction_date,due_date_fixed,end_rollover_date,dpd_90_date,days_past_due,transaction_date,amount,transaction_by_due_date,transaction_by_end_rollover_date,transaction_by_dpd_90_date
0,16296,7529969,7,4300.000,2022-09-12,2022-01-20,4446.270,4446.270,2022-01-27,2022-01-27,2022-01-27,2022-01-27,2022-01-30,2022-04-30,0,2022-01-27,4446.200,yes,yes,yes
1,16357,840549,21,56000.000,2022-09-12,2022-01-21,60872.010,60872.010,2022-02-11,2022-02-11,2022-02-11,2022-02-11,2022-02-16,2022-05-17,0,2022-02-11,60872.000,yes,yes,yes


In [ ]:
df2.shape

(387457, 20)

In [ ]:
trans_by_due_date = df2[df2['transaction_by_due_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_trans_by_due_date = trans_by_due_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_trans_by_due_date = amount_trans_by_due_date.rename(columns={'amount':'amount_trans_by_due_date'})

amount_trans_by_due_date.head(2)

,loan_mifos_id,amount_trans_by_due_date
0,3697,155.100
1,3698,138.700


In [ ]:
amount_trans_by_due_date.shape

(152548, 2)

In [ ]:
transaction_by_end_rollover_date = df2[df2['transaction_by_end_rollover_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_transaction_by_end_rollover_date = transaction_by_end_rollover_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_transaction_by_end_rollover_date = amount_transaction_by_end_rollover_date.rename(columns={'amount':'amount_transaction_by_end_rollover_date'})

amount_transaction_by_end_rollover_date.head(2)

,loan_mifos_id,amount_transaction_by_end_rollover_date
0,3697,155.100
1,3698,138.700


In [ ]:
amount_transaction_by_end_rollover_date.shape

(169899, 2)

In [ ]:
transaction_by_dpd_90_date = df2[df2['transaction_by_dpd_90_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_transaction_by_dpd_90_date = transaction_by_dpd_90_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_transaction_by_dpd_90_date = amount_transaction_by_dpd_90_date.rename(columns={'amount':'amount_transaction_by_dpd_90_date'})

amount_transaction_by_dpd_90_date.head(2)

,loan_mifos_id,amount_transaction_by_dpd_90_date
0,3697,155.100
1,3698,138.700


In [ ]:
amount_transaction_by_dpd_90_date.shape

(185731, 2)

In [ ]:
final_df = pd.merge(amount_trans_by_due_date, amount_transaction_by_end_rollover_date, on = 'loan_mifos_id', how = 'outer')
final_df = pd.merge(final_df, amount_transaction_by_dpd_90_date, on = 'loan_mifos_id', how = 'outer')
final_df = final_df.fillna(0)

final_df.head(2)

,loan_mifos_id,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date
0,3697,155.100,155.100,155.100
1,3698,138.700,138.700,138.700


In [ ]:
df_loans = df[['loan_mifos_id', 'store_number', 'term_frequency', 'principal_disbursed', 'curr_time', 'disbursed_on_date', 'expected_matured_on_date', 'end_rollover_date', 'dpd_90_date', 'total_expected_repayment']]
final_df = pd.merge(df_loans, final_df, on = 'loan_mifos_id', how = 'left')
final_df = final_df.fillna(0)

final_df.head(2)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,curr_time,disbursed_on_date,expected_matured_on_date,end_rollover_date,dpd_90_date,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date
0,16296,7529969,7,4300.000,2022-09-12,2022-01-20,2022-01-27,2022-01-30,2022-04-30,4446.270,4446.200,4446.200,4446.200
1,16357,840549,21,56000.000,2022-09-12,2022-01-21,2022-02-11,2022-02-16,2022-05-17,60872.010,60872.000,60872.000,60872.000


In [ ]:
final_df.shape

(198312, 13)

In [ ]:
#final_df['transaction_date'] = pd.to_datetime(final_df['transaction_date'], errors='coerce')

final_df['disbursed_on_date'] = pd.to_datetime(final_df['disbursed_on_date'], errors='coerce')

final_df['expected_matured_on_date'] = pd.to_datetime(final_df['expected_matured_on_date'], errors='coerce')

final_df['end_rollover_date'] = pd.to_datetime(final_df['end_rollover_date'], errors='coerce')

final_df['dpd_90_date'] = pd.to_datetime(final_df['dpd_90_date'], errors='coerce')

final_df['curr_time'] = pd.to_datetime(final_df['curr_time'], errors='coerce')

In [ ]:
final_df['days_past_due_date'] = (final_df['curr_time'] - final_df['expected_matured_on_date']).dt.days

final_df['days_past_end_rollover_date'] = (final_df['curr_time'] - final_df['end_rollover_date']).dt.days

final_df['days_past_dpd_90_date'] = (final_df['curr_time'] - final_df['dpd_90_date']).dt.days

final_df.head()

,loan_mifos_id,store_number,term_frequency,principal_disbursed,curr_time,disbursed_on_date,expected_matured_on_date,end_rollover_date,dpd_90_date,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,days_past_due_date,days_past_end_rollover_date,days_past_dpd_90_date
0,16296,7529969,7,4300.000,2022-09-12,2022-01-20,2022-01-27,2022-01-30,2022-04-30,4446.270,4446.200,4446.200,4446.200,228,225,135
1,16357,840549,21,56000.000,2022-09-12,2022-01-21,2022-02-11,2022-02-16,2022-05-17,60872.010,60872.000,60872.000,60872.000,213,208,118
2,16386,7216519,7,1400.000,2022-09-12,2022-01-21,2022-01-28,2022-01-31,2022-05-01,1447.670,1447.600,1447.600,1447.600,227,224,134
3,16406,608561,21,20000.000,2022-09-12,2022-01-21,2022-02-11,2022-02-16,2022-05-17,21740.010,21740.000,21740.000,21740.000,213,208,118
4,16655,7445841,7,2700.000,2022-09-12,2022-01-21,2022-01-28,2022-01-31,2022-05-01,2791.870,2791.800,2791.800,2791.800,227,224,134


In [ ]:
total_principal_disbursed = final_df.groupby(['store_number'], as_index=False)['principal_disbursed'].sum()
total_principal_disbursed = total_principal_disbursed.rename(columns={'principal_disbursed':'total_principal_disbursed'})

total_principal_disbursed.head(2)

,store_number,total_principal_disbursed
0,000030,55600.000
1,003215,270000.000


In [ ]:
repayments_by_due_date = final_df.groupby(['store_number'], as_index=False)['amount_trans_by_due_date'].sum()
repayments_by_due_date = repayments_by_due_date.rename(columns={'amount_trans_by_due_date':'total_repayments_by_due_date'})

repayments_by_due_date.head(2)

,store_number,total_repayments_by_due_date
0,000030,46141.760
1,003215,290520.000


In [ ]:
repayments_by_end_rollover_date = final_df.groupby(['store_number'], as_index=False)['amount_transaction_by_end_rollover_date'].sum()
repayments_by_end_rollover_date = repayments_by_end_rollover_date.rename(columns={'amount_transaction_by_end_rollover_date':'total_repayments_by_end_rollover_date'})

repayments_by_end_rollover_date.head(2)

,store_number,total_repayments_by_end_rollover_date
0,000030,46141.760
1,003215,290520.000


In [ ]:
final = pd.merge(total_principal_disbursed, repayments_by_due_date, on = 'store_number', how = 'outer')
final = pd.merge(final, repayments_by_end_rollover_date, on = 'store_number', how = 'outer')
final = final.fillna(0)

final.head(2)

,store_number,total_principal_disbursed,total_repayments_by_due_date,total_repayments_by_end_rollover_date
0,000030,55600.000,46141.760,46141.760
1,003215,270000.000,290520.000,290520.000


In [ ]:
final_df['Repayments by Due Date as a %age of Principal amount'] = final_df['amount_trans_by_due_date'] / final_df['principal_disbursed']
final_df['Repayments by Rollover End Date as a %age of Principal amount'] = final_df['amount_transaction_by_end_rollover_date'] / final_df['principal_disbursed']
final_df['Repayments by DPD90 as a %age of Principal'] = final_df['amount_transaction_by_dpd_90_date'] / final_df['principal_disbursed']

final_df.head(10)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,curr_time,disbursed_on_date,expected_matured_on_date,end_rollover_date,dpd_90_date,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,days_past_due_date,days_past_end_rollover_date,days_past_dpd_90_date,Repayments by Due Date as a %age of Principal amount,Repayments by Rollover End Date as a %age of Principal amount,Repayments by DPD90 as a %age of Principal
0,16296,7529969,7,4300.000,2022-09-12,2022-01-20,2022-01-27,2022-01-30,2022-04-30,4446.270,4446.200,4446.200,4446.200,228,225,135,1.034,1.034,1.034
1,16357,840549,21,56000.000,2022-09-12,2022-01-21,2022-02-11,2022-02-16,2022-05-17,60872.010,60872.000,60872.000,60872.000,213,208,118,1.087,1.087,1.087
2,16386,7216519,7,1400.000,2022-09-12,2022-01-21,2022-01-28,2022-01-31,2022-05-01,1447.670,1447.600,1447.600,1447.600,227,224,134,1.034,1.034,1.034
3,16406,608561,21,20000.000,2022-09-12,2022-01-21,2022-02-11,2022-02-16,2022-05-17,21740.010,21740.000,21740.000,21740.000,213,208,118,1.087,1.087,1.087
4,16655,7445841,7,2700.000,2022-09-12,2022-01-21,2022-01-28,2022-01-31,2022-05-01,2791.870,2791.800,2791.800,2791.800,227,224,134,1.034,1.034,1.034
5,16048,7141126,7,3400.000,2022-09-12,2022-01-20,2022-01-27,2022-01-30,2022-04-30,3515.670,3515.600,3515.600,3515.600,228,225,135,1.034,1.034,1.034
6,16166,922142,21,10000.000,2022-09-12,2022-01-20,2022-02-10,2022-02-15,2022-05-16,11777.200,1500.000,3500.000,11777.200,214,209,119,0.150,0.350,1.178
7,16185,7831031,7,1000.000,2022-09-12,2022-01-20,2022-01-27,2022-01-30,2022-04-30,1034.010,1034.000,1034.000,1034.000,228,225,135,1.034,1.034,1.034
8,16328,7117860,7,1600.000,2022-09-12,2022-01-20,2022-01-27,2022-01-30,2022-04-30,1654.410,1654.400,1654.400,1654.400,228,225,135,1.034,1.034,1.034
9,16605,7022294,7,9000.000,2022-09-12,2022-01-21,2022-01-28,2022-01-31,2022-05-01,9306.070,9306.000,9306.000,9306.000,227,224,134,1.034,1.034,1.034


In [ ]:
matured_loans = final_df[final_df['days_past_due_date'] >= 0]

average_repayments_by_due_date = matured_loans.groupby(['store_number'], as_index=False)['Repayments by Due Date as a %age of Principal amount'].mean()
average_repayments_by_due_date = average_repayments_by_due_date.rename(columns={'Repayments by Due Date as a %age of Principal amount':'Average Repayments by Due Date as a %age of Principal amount'})

matured_loans_96_above = average_repayments_by_due_date[(average_repayments_by_due_date['Average Repayments by Due Date as a %age of Principal amount'] >= 0.96)]

#matured_loans_96_above = matured_loans_96_above[['loan_mifos_id', 'store_number']]

matured_loans_96_above.head()

,store_number,Average Repayments by Due Date as a %age of Principal amount
0,000030,1.027
1,003215,1.076
3,058133,1.078
4,105295,1.087
12,110970,0.977


In [ ]:
rollovered_loans = final_df[final_df['days_past_end_rollover_date'] >= 0]

average_repayments_by_due_date = rollovered_loans.groupby(['store_number'], as_index=False)['Repayments by Rollover End Date as a %age of Principal amount'].mean()
average_repayments_by_due_date = average_repayments_by_due_date.rename(columns={'Repayments by Rollover End Date as a %age of Principal amount':'Average Repayments by Rollover End Date as a %age of Principal amount'})

rollovered_loans_102_above = average_repayments_by_due_date[average_repayments_by_due_date['Average Repayments by Rollover End Date as a %age of Principal amount'] >= 1.02]

rollovered_loans_102_above.head()

,store_number,Average Repayments by Rollover End Date as a %age of Principal amount
0,000030,1.027
1,003215,1.076
3,058133,1.078
4,105295,1.087
11,110901,1.049


In [ ]:
final = pd.merge(matured_loans_96_above, rollovered_loans_102_above, on = 'store_number', how = 'outer')

final = final.fillna(0)

final = final[(final['Average Repayments by Due Date as a %age of Principal amount'] >= 0.96) & (final['Average Repayments by Due Date as a %age of Principal amount'] >= 1.02)]

final.head(5)

,store_number,Average Repayments by Due Date as a %age of Principal amount,Average Repayments by Rollover End Date as a %age of Principal amount
0,000030,1.027,1.027
1,003215,1.076,1.076
2,058133,1.078,1.078
3,105295,1.087,1.087
5,111160,1.035,1.035


In [ ]:
final_list = list(final["store_number"].unique())

In [ ]:
df = pd.read_excel("/content/Limits_refresh_summary_20220922_multiple_products_2.xlsx")
#df = pd.read_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220908_multiple_products_rec.xlsx")
df = df.rename(columns={'final_21_limit':'old_final_21_limit', 'final_7_limit':'old_final_7_limit', 'final_1_limit':'old_final_1_limit'})

df[['old_final_21_limit', 'old_final_7_limit', 'old_final_1_limit']].sum()

old_final_21_limit    353222300
old_final_7_limit     480198200
old_final_1_limit     482168800
dtype: int64

In [ ]:
df2 = pd.read_excel("/content/Limits_refresh_summary_20220922_multiple_products_rec_new.xlsx")
#df = pd.read_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220908_multiple_products_rec.xlsx")
df2 = df2.rename(columns={'new_final_21_limit':'new_final_21_limit_1', 'updated_final_7_limit':'new_final_7_limit_1', 'updated_final_1_limit':'new_final_1_limit_1'})
df2 = df2[['store_number', 'new_final_21_limit_1', 'new_final_7_limit_1', 'new_final_1_limit_1', 'old_src_crdt_score']]

df2[['new_final_21_limit_1', 'new_final_7_limit_1', 'new_final_1_limit_1']].sum()

new_final_21_limit_1    204942500
new_final_7_limit_1     372819700
new_final_1_limit_1     374488300
dtype: int64

In [ ]:
combined = pd.merge(df, df2, on = 'store_number', how = 'left')

print(combined[['old_final_21_limit', 'old_final_7_limit', 'old_final_1_limit']].sum())

print(combined[['new_final_21_limit_1', 'new_final_7_limit_1', 'new_final_1_limit_1']].sum())

old_final_21_limit    353222300
old_final_7_limit     480198200
old_final_1_limit     482168800
dtype: int64
new_final_21_limit_1    204942500
new_final_7_limit_1     372819700
new_final_1_limit_1     374488300
dtype: int64


In [ ]:
def calc_zeroized_7(df):
    
    old_final_7_limit = df['old_final_7_limit']
    new_final_7_limit_1 = df['new_final_7_limit_1']
    
    if new_final_7_limit_1 == 0 and old_final_7_limit > 0:
        return 1
    else:
        return 0
    

def calc_zeroized_1(df):
    
    old_final_1_limit = df['old_final_1_limit']
    new_final_1_limit_1 = df['new_final_7_limit_1']
    
    if new_final_1_limit_1 == 0 and old_final_1_limit > 0:
        return 1
    else:
        return 0

In [ ]:
combined['zeroized_7'] = combined.apply(lambda x: calc_zeroized_7(x), axis = 1)
combined['zeroized_1'] = combined.apply(lambda x: calc_zeroized_1(x), axis = 1)

print(combined['zeroized_7'].value_counts())
print(combined['zeroized_1'].value_counts())

0    55982
1     4251
Name: zeroized_7, dtype: int64
0    52694
1     7539
Name: zeroized_1, dtype: int64


In [ ]:
combined['store_number'] = combined['store_number'].astype(str)

In [ ]:
final_combined = pd.merge(combined, final, on='store_number', how='left')

final_combined['Average Repayments by Due Date as a %age of Principal amount'] = final_combined['Average Repayments by Due Date as a %age of Principal amount'].fillna(0)
final_combined['Average Repayments by Rollover End Date as a %age of Principal amount'] = final_combined['Average Repayments by Rollover End Date as a %age of Principal amount'].fillna(0)

final_combined.head()

,Unnamed: 0,mobile_number,store_number,sum_3_months_trx_val,avg_3_months_trx_val,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,blacklist_flag,total_final_21_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_version,created_at,record_added_to_warehouse_on_timestamp,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,old_src_crdt_score,zeroized_7,zeroized_1,Average Repayments by Due Date as a %age of Principal amount,Average Repayments by Rollover End Date as a %age of Principal amount
0,0,254723173634,786152,5430.000,1810.000,3700.000,2022-08-25,2022-09-20,27,4,0.150,No_rules_relaxed,2.000,Yes,1.000,494024.000,Approve,0.000,True,NaN,1,601.000,30.000,12698.000,9206.000,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.000,1.000,written-off_default,NaN,12698.000,2017-12-18,0.000,0.000,1738.000,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 1,Band 2,0.000,0.000,0.000,0,0,2,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,1,0,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,NaN,0,0,0.000,0.000
1,1,254723916436,165978,2454577.510,818192.503,1005490.230,2022-08-23,2022-09-21,30,30,1.000,No_rules_relaxed,1.000,Yes,1.000,20417564.000,Approve,0.000,True,NaN,1,601.000,30.000,20000.000,12800.000,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.000,1.000,written-off_default,NaN,20000.000,2017-12-18,0.000,0.000,1738.000,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 7,Band 2,0.250,0.150,0.150,0,1,1,0.500,0.125,0.075,0.075,102274.063,61364.438,61364.438,0.000,61364.438,61364.438,0,0,0,1,304500,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,NaN,0,0,0.000,0.000
2,2,254720475133,745142,463108.240,154369.413,194218.800,2022-08-22,2022-09-21,31,30,0.970,No_rules_relaxed,1.000,Yes,1.000,22893884.000,Reject,0.000,True,NaN,1,601.000,30.000,28684.000,0.000,2017-12-21,2018-01-20,2019-12-31,2018-01-20,710.000,1.000,written-off_default,NaN,28684.000,2017-12-21,0.000,0.000,1735.000,0,0,0.350,0.120,0.120,0.700,0.706,0.706,Band 7,Band 2,0.250,0.150,0.150,0,1,1,0.500,0.087,0.053,0.053,13507.324,8172.498,8172.498,0.000,8172.498,8172.498,0,0,0,1,27200,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,NaN,0,0,0.000,0.000
3,3,254724756423,978161,1100.000,366.667,0.000,NaT,NaT,0,0,0.000,No_rules_relaxed,NaN,0,0.000,13261352.000,Approve,0.000,True,NaN,2,601.000,30.000,28600.000,0.000,2018-01-07,2018-02-06,2018-12-31,2018-02-06,328.000,1.000,written-off_default,NaN,28680.000,2017-12-19,1.000,0.500,1718.000,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 1,Band 2,0.000,0.000,0.000,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,1,0,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,NaN,0,0,0.000,0.000
4,4,254722990261,510261,1232666.710,410888.903,668797.530,2022-08-22,2022-09-21,31,

In [ ]:
def updated_7_limit(df):
    
    zeroized_7 = df['zeroized_7']
    old_final_7_limit = df['old_final_7_limit']
    new_final_7_limit_1 = df['new_final_7_limit_1']
    average_repayments_by_due_date = df['Average Repayments by Due Date as a %age of Principal amount']
    average_repayments_by_rollover_end_date = df['Average Repayments by Rollover End Date as a %age of Principal amount']
    
    if zeroized_7 == 1 and average_repayments_by_due_date >= 0.96 and average_repayments_by_rollover_end_date >= 1.02:
        return old_final_7_limit
    else:
        return new_final_7_limit_1
    
    
def updated_1_limit(df):
    
    zeroized_1 = df['zeroized_1']
    old_final_1_limit = df['old_final_1_limit']
    new_final_1_limit_1 = df['new_final_1_limit_1']
    average_repayments_by_due_date = df['Average Repayments by Due Date as a %age of Principal amount']
    average_repayments_by_rollover_end_date = df['Average Repayments by Rollover End Date as a %age of Principal amount']
    
    if zeroized_1 == 1 and average_repayments_by_due_date >= 0.96 and average_repayments_by_rollover_end_date >= 1.02:
        return old_final_1_limit
    else:
        return new_final_1_limit_1

In [ ]:
final_combined['updated_7_limit'] = final_combined.apply(lambda x: updated_7_limit(x), axis = 1)
final_combined['updated_1_limit'] = final_combined.apply(lambda x: updated_1_limit(x), axis = 1)

print(final_combined[['new_final_21_limit_1', 'updated_7_limit', 'updated_1_limit']].sum())

new_final_21_limit_1    204942500
updated_7_limit         427187800
updated_1_limit         428956600
dtype: int64


In [ ]:
final_combined.to_excel("Limits_refresh_summary_20220914_multiple_products_rec_1_new.xlsx")